(8)=
# Chapter 8: Programming Practices

**Topics Covered:**
- Code documentation and commenting
- Testing and validation
- Error handling and exceptions
- Debugging techniques

---

Opening
Alright everyone, welcome to Chapter 8. Today we're talking about programming practices — documentation, testing, error handling, and debugging. This might sound like housekeeping compared to writing actual calculations, but honestly, this is what separates code that works from code you can trust. In chemical engineering, if your reactor sizing calculation is wrong and nobody catches it, that's not just a bad grade — in industry, that's a safety incident.

---

(8.1)=
## 8.1 Code Documentation and Commenting

---

Let's start with documentation. You've all written code that you look at a week later and think "what was I doing here?" That's what documentation prevents.

---

Good documentation makes code understandable, maintainable, and reusable. In chemical engineering, where calculations can be complex and involve many parameters, clear documentation is critical.

(8.1.1)=
### 8.1.1 Docstrings

Docstrings are special comments that describe what a function does, its parameters, and return values.

---

Python has a built-in convention for documenting functions called docstrings. It's a string that goes right at the top of your function, inside triple quotes.

Let me show you an example — here we have a function that calculates the Reynolds number. Look at the docstring. It tells you what the function does, what each parameter means with units, what it returns, and even gives you an example. There's also a Notes section explaining the flow regime thresholds.

[Run the reynolds_number cell]

Notice I can actually access this docstring programmatically with reynolds_number.__doc__. This is what Python reads when you call help() on a function. So if someone imports your code into their project, they can see exactly how to use it without reading your source code.

The format I'm using here is called NumPy-style docstrings — it's the standard in scientific Python. Parameters, Returns, Examples, Notes — those are all standard sections.

---

In [1]:
import math

def reynolds_number(rho, v, D, mu):
    """
    Calculate the Reynolds number for fluid flow in a pipe.
    
    The Reynolds number is a dimensionless quantity that predicts
    flow patterns in different fluid flow situations.
    
    Parameters
    ----------
    rho : float
        Fluid density (kg/m³)
    v : float
        Flow velocity (m/s)
    D : float
        Pipe diameter (m)
    mu : float
        Dynamic viscosity (Pa·s)
    
    Returns
    -------
    float
        Reynolds number (dimensionless)
    
    Examples
    --------
    >>> reynolds_number(1000, 2.0, 0.1, 0.001)
    200000.0
    
    Notes
    -----
    Flow regimes:
    - Re < 2300: Laminar flow
    - 2300 ≤ Re ≤ 4000: Transitional flow
    - Re > 4000: Turbulent flow
    """
    Re = (rho * v * D) / mu
    return Re

# Test the function
Re = reynolds_number(1000, 2.0, 0.1, 0.001)
print(f"Reynolds number: {Re:.0f}")

# Access the docstring
print("\nFunction documentation:")
print(reynolds_number.__doc__)

Reynolds number: 200000

Function documentation:

    Calculate the Reynolds number for fluid flow in a pipe.
    
    The Reynolds number is a dimensionless quantity that predicts
    flow patterns in different fluid flow situations.
    
    Parameters
    ----------
    rho : float
        Fluid density (kg/m³)
    v : float
        Flow velocity (m/s)
    D : float
        Pipe diameter (m)
    mu : float
        Dynamic viscosity (Pa·s)
    
    Returns
    -------
    float
        Reynolds number (dimensionless)
    
    Examples
    --------
    >>> reynolds_number(1000, 2.0, 0.1, 0.001)
    200000.0
    
    Notes
    -----
    Flow regimes:
    - Re < 2300: Laminar flow
    - 2300 ≤ Re ≤ 4000: Transitional flow
    - Re > 4000: Turbulent flow
    


In [7]:
import math, os

print(math.pi.__doc__)
print(math.sqrt.__doc__)
print(math.sin.__doc__)
print(math.cos.__doc__)
print(os.path.__doc__)

Convert a string or number to a floating point number, if possible.
Return the square root of x.
Return the sine of x (measured in radians).
Return the cosine of x (measured in radians).
Common operations on Posix pathnames.

Instead of importing this module directly, import os and refer to
this module as os.path.  The "os.path" name is an alias for this
module on Posix systems; on other systems (e.g. Windows),
os.path provides the same operations in a manner specific to that
platform, and is an alias to another module (e.g. ntpath).

Some of this can actually be useful on non-Posix systems too, e.g.
for manipulation of the pathname component of URLs.



(8.1.2)=
### 8.1.2 Inline Comments

---

Now, docstrings describe the function as a whole. Inline comments explain specific lines. But here's the key rule: comment the why, not the what.

Look at this Arrhenius rate constant function. I don't need a comment that says # multiply A by exponential — you can see that from the code. But the comment # Negative sign because higher Ea means slower reaction — that explains the physics behind the math. That's useful.

Same with the test values below: # Using typical values for a first-order decomposition reaction. That tells the reader why we picked these numbers, not just what they are.

---

Use comments to explain **why** you're doing something, not **what** you're doing (the code shows what).

In [ ]:
def arrhenius_rate_constant(A, Ea, T):
    """
    Calculate reaction rate constant using Arrhenius equation.
    
    Parameters
    ----------
    A : float
        Pre-exponential factor (1/s)
    Ea : float
        Activation energy (J/mol)
    T : float
        Temperature (K)
    
    Returns
    -------
    float
        Rate constant k (1/s)
    """
    R = 8.314  # J/(mol·K) - Universal gas constant
    
    # Negative sign because higher Ea means slower reaction
    exponent = -Ea / (R * T)
    
    k = A * math.exp(exponent)
    
    return k

# Good comment: explains WHY we use these specific values
# Using typical values for a first-order decomposition reaction
A = 1.0e10  # s⁻¹
Ea = 50000  # J/mol
T = 298.15  # K (25°C, standard conditions)

k = arrhenius_rate_constant(A, Ea, T)
print(f"Rate constant at {T} K: {k:.4e} s⁻¹")

(8.1.3)=
### 8.1.3 Code Organization Best Practices

---

Here's a side-by-side comparison. Look at the first function: calc(x, y, z). What does that compute? You have no idea. Could be anything.

Now look at gravitational_force(mass1_kg, mass2_kg, distance_m). Immediately clear. The variable names include units. There's a docstring. Someone reading this six months from now — including future you — knows exactly what's going on.

[Run the cell]

They give the same numerical answer, but one is maintainable code and the other is a time bomb waiting for someone to pass arguments in the wrong order.

---

In [ ]:
# BAD: Unclear variable names, no documentation
def calc(x, y, z):
    return x * y / z**2

# GOOD: Clear names, documentation, units specified
def gravitational_force(mass1_kg, mass2_kg, distance_m):
    """
    Calculate gravitational force between two masses.
    
    Parameters
    ----------
    mass1_kg : float
        Mass of first object (kg)
    mass2_kg : float
        Mass of second object (kg)
    distance_m : float
        Distance between centers (m)
    
    Returns
    -------
    float
        Gravitational force (N)
    """
    G = 6.674e-11  # N·m²/kg² - Gravitational constant
    
    force_N = G * mass1_kg * mass2_kg / (distance_m ** 2)
    
    return force_N

# Test both versions
F1 = calc(100, 50, 1.0)
F2 = gravitational_force(100, 50, 1.0)

print(f"Both give same result: {F1:.6e} N")
print("But the second is much more readable!")

(8.2)=
## 8.2 Testing and Validation

Testing ensures your code produces correct results. For chemical engineering calculations, errors can have serious consequences.

---

Alright, so your code is well-documented. How do you know it's correct?


---

(8.2.1)=
### 8.2.1 Unit Testing with Known Results

---

The idea is simple: you know the answer to certain cases, so test against them. Here we have the ideal gas law: P = nRT/V.

Look at test_ideal_gas_pressure. I have three tests:

Test 1: A specific numerical case — 50 moles at 373.15 K in 0.5 m³. I calculated what P should be ahead of time and I check that my function matches.

Test 2: Another known case at different conditions.

Test 3: A scaling check — if I double the moles, pressure should double. This tests the physics, not just one number.

---

In [9]:
def ideal_gas_pressure(n, T, V):
    """
    Calculate pressure using ideal gas law: P = nRT/V
    
    Parameters
    ----------
    n : float
        Amount of gas (mol)
    T : float
        Temperature (K)
    V : float
        Volume (m³)
    
    Returns
    -------
    float
        Pressure (Pa)
    """
    R = 8.314  # J/(mol·K)
    P = (n * R * T) / V
    return P

# Test with known result: 1 mol at STP should give ~101325 Pa
def test_ideal_gas_pressure():
    # Test 1: Known textbook example
    P1 = ideal_gas_pressure(n=50.0, T=373.15, V=0.5)
    print(f"Calculated Pressure: {P1:.2f} Pa")
    assert abs(P1 - 31097.41) < 1.0, "Test 1 failed"

    # Test 2: Room temperature gas
    P2 = ideal_gas_pressure(n=1.0, T=298.15, V=1.0)
    print(f"Calculated Pressure: {P2:.2f} Pa")
    assert abs(P2 - 2478.9) < 1.0, "Test 2 failed"

    # Test 3: Scaling check (double moles → double pressure)
    P3 = ideal_gas_pressure(n=2.0, T=300.0, V=1.0)
    P4 = ideal_gas_pressure(n=1.0, T=300.0, V=1.0)
    assert abs(P3 - 2 * P4) < 1e-6, "Test 3 failed"

    print("All ideal gas law tests passed!")


# Run tests
test_ideal_gas_pressure()

Calculated Pressure: 310236.91 Pa


AssertionError: Test 1 failed

(8.2.2)=
### 8.2.2 Boundary Testing

---

Beyond normal cases, test the edges. What happens with a very small volume? Pressure should be huge. What about zero moles? Pressure should be zero.

---

In [10]:
def test_edge_cases():
    # Very small volume → large pressure
    P = ideal_gas_pressure(1.0, 300.0, 1e-3)
    assert P > 1e6

    # Zero moles → zero pressure
    P = ideal_gas_pressure(0.0, 300.0, 1.0)
    assert P == 0.0

    print("Edge case tests passed!")

# Run edge case tests
test_edge_cases()

Edge case tests passed!


(8.3)=
## 8.3 Error Handling

Robust code anticipates and handles errors gracefully. This prevents crashes and provides useful feedback.

---

Now, what happens when things go wrong? Your code will eventually get bad inputs — a user types a string instead of a number, a sensor returns zero, a file is missing. You need to handle that gracefully, not with a cryptic crash.

---

(8.3.1)=
### 8.3.1 Try-Except Blocks

In [9]:
def safe_division(numerator, denominator):
    """
    Safely divide two numbers with error handling.
    
    Parameters
    ----------
    numerator : float
    denominator : float
    
    Returns
    -------
    float or None
        Result of division, or None if error
    """
    try:
        result = numerator / denominator
        return result
    except ZeroDivisionError:
        print(f"Error: Cannot divide {numerator} by zero!")
        return None
    except TypeError:
        print(f"Error: Invalid input types: {type(numerator)}, {type(denominator)}")
        return None

# Test error handling
print("Testing error handling:\n")

result1 = safe_division(10, 2)
print(f"10 / 2 = {result1}")

result2 = safe_division(10, 0)
print(f"10 / 0 = {result2}")

result3 = safe_division(10, "hello")
print(f"10 / 'hello' = {result3}")

Testing error handling:

10 / 2 = 5.0
Error: Cannot divide 10 by zero!
10 / 0 = None
Error: Invalid input types: <class 'int'>, <class 'str'>
10 / 'hello' = None


### 8.3.2 Input Validation

In [10]:
def heat_capacity_calculation(mass, Cp, delta_T):
    """
    Calculate heat required: Q = m × Cp × ΔT
    
    Parameters
    ----------
    mass : float
        Mass (kg), must be positive
    Cp : float
        Specific heat capacity (J/kg·K), must be positive
    delta_T : float
        Temperature change (K), can be negative for cooling
    
    Returns
    -------
    float
        Heat energy (J)
    
    Raises
    ------
    ValueError
        If mass or Cp is not positive
    """
    # Validate inputs
    if mass <= 0:
        raise ValueError(f"Mass must be positive, got {mass} kg")
    
    if Cp <= 0:
        raise ValueError(f"Heat capacity must be positive, got {Cp} J/kg·K")
    
    # Calculate heat
    Q = mass * Cp * delta_T
    
    return Q

In [11]:
# Valid input
try:
    Q = heat_capacity_calculation(2.5, 4184, 50)
    print(f"✓ Valid: Q = {Q:.2f} J")
except ValueError as e:
    print(f"✗ {e}")


✓ Valid: Q = 523000.00 J


In [12]:
# Invalid mass
try:
    Q = heat_capacity_calculation(-2.5, 4184, 50)
    print(f"✓ Should not reach here")
except ValueError as e:
    print(f"✓ Caught error: {e}")


✓ Caught error: Mass must be positive, got -2.5 kg


(8.3.3)=
### 8.3.3 Graceful Degradation

In [13]:
def average_temperature(temperatures):
    """
    Calculate the average temperature from a list.
    Invalid entries are skipped.
    
    Parameters
    ----------
    temperatures : list
        List of temperature values (°C)
    
    Returns
    -------
    float
        Average temperature of valid values
    """
    total = 0
    count = 0

    for temp in temperatures:
        try:
            temp_value = float(temp)

            if temp_value < -273.15:
                raise ValueError("Below absolute zero")

            total += temp_value
            count += 1

        except (ValueError, TypeError):
            continue  # Skip invalid values

    if count == 0:
        return None

    return total / count

In [14]:
# Test data with valid and invalid values
temps = [22.5, 25.0, "hot", -300, 18.7, None, 20.1]

avg = average_temperature(temps)

if avg is not None:
    print(f"Average temperature: {avg:.2f} °C")
else:
    print("No valid temperature data found.")

Average temperature: 21.58 °C


(8.4)=
## 8.4 Debugging Techniques

Debugging is the process of finding and fixing errors in code. Systematic debugging saves time and frustration.

---

Alright, you've written tests and they're failing. Now what? Time to debug.

---

(8.4.1)=
### 8.4.1 Print Statement Debugging

---

The simplest technique: add print statements to see what's happening inside your function.

[Run the cell]

See the DEBUG: lines? They show you the values of C_initial, C_final, and the computed conversion at each step. This is quick and dirty, but it works. Just remember to remove your debug prints before sharing your code.

---

In [ ]:
def calculate_conversion(C_initial, C_final):
    """
    Calculate conversion: X = (C0 - C) / C0
    """
    print(f"DEBUG: C_initial = {C_initial}, C_final = {C_final}")  # Debug print
    
    conversion = (C_initial - C_final) / C_initial
    
    print(f"DEBUG: conversion = {conversion}")  # Debug print
    
    return conversion

# Test
X = calculate_conversion(5.0, 2.0)
print(f"\nFinal conversion: {X:.2%}")

(8.4.2)=
### 8.4.2 Assert Statements for Assumptions

---

We saw assert in testing, but you can also use it inside functions to check your assumptions as the code runs.

In Python, assert is a debugging statement used to check that a condition is true during program execution.

- If the condition is True → nothing happens.
- If the condition is False → Python raises an 

In calculate_residence_time, the asserts say: volume must be positive, flow rate must be positive, and the result should be positive. If any of these fail, the code stops immediately and tells you which assumption was violated.

[Run the cell]

Valid inputs: residence time is 20 minutes. Negative volume: the assert fires and says "Volume must be positive, got -100." This catches bugs early, right at the source, instead of letting bad values propagate through your calculation.

---

In [ ]:
def calculate_residence_time(volume, flow_rate):
    """
    Calculate residence time: τ = V / Q
    
    Parameters
    ----------
    volume : float
        Reactor volume (L)
    flow_rate : float
        Volumetric flow rate (L/min)
    
    Returns
    -------
    float
        Residence time (min)
    """
    # Assert our assumptions
    assert volume > 0, f"Volume must be positive, got {volume}"
    assert flow_rate > 0, f"Flow rate must be positive, got {flow_rate}"
    
    tau = volume / flow_rate
    
    # Check the result makes sense
    assert tau > 0, "Residence time should be positive"
    
    return tau

# Valid case
try:
    tau = calculate_residence_time(100, 5)
    print(f"✓ Residence time: {tau} min")
except AssertionError as e:
    print(f"✗ Assertion failed: {e}")

# Invalid case
try:
    tau = calculate_residence_time(-100, 5)
    print(f"✓ Residence time: {tau} min")
except AssertionError as e:
    print(f"✓ Caught invalid input: {e}")

(8.4.3)=
### 8.4.3 Usage of breakpoint()

**breakpoint()** is a built-in Python function that pauses program execution at a specific line and enters interactive debugging mode. It allows you to inspect variable values, step through code line by line, and identify logic or runtime errors while the program is running. This is useful for understanding how a program behaves and for locating bugs without adding multiple print statements.


Run ```kelvin.py```, and check how breakpoint() works.


---

Now for the most powerful debugging tool. breakpoint() pauses your program and drops you into an interactive debugger where you can inspect every variable, step line by line, and figure out exactly what's going wrong.

Let me show you kelvin.py. This is a simple function that converts Celsius to Kelvin, but it has a bug — it subtracts 273.15 instead of adding it. There's a breakpoint() right after the conversion line.

When you run this script from the terminal with python kelvin.py, it will pause at the breakpoint. You can type temp to see the current Celsius value, type kelvin to see the (wrong) converted value, and immediately spot that 20°C became -253.15 K instead of 293.15 K. You type c to continue or q to quit.

This is much faster than adding a dozen print statements. For anything more than a trivial bug, breakpoint() is the way to go.



---

## Summary

In this chapter, you learned:

1. **Documentation**
   - Writing comprehensive docstrings
   - Using comments effectively
   - Naming conventions and code organization

2. **Testing**
   - Unit testing with known results
   - Boundary condition testing
   - Conservation law verification

3. **Error Handling**
   - Try-except blocks
   - Input validation
   - Graceful degradation

4. **Debugging**
   - Print statement debugging
   - Assert statements
   - Step-by-step verification

5. **Best Practices**
   - Professional code style
   - Comprehensive documentation
   - Robust error handling
   - Systematic testing

**Remember**: Good programming practices prevent errors, save debugging time, and make your code understandable to others (including your future self!).